In [50]:
import sympy as sp

A = sp.Matrix([
    [1, 7, 3],
    [1/7, 1, 1/3],
    [1/3, 3, 1]
])

lambda_sym = sp.symbols('lambda')

# matrix (A - lambda * I)
A_lambda_I = A - lambda_sym * sp.eye(3)
determinant = A_lambda_I.det()

# solving the characteristic equation det(A - lambda * I) = 0 for eigenvalues
eigenvalues = sp.solve(determinant, lambda_sym)
eigenvalues


[3.00702176512993,
 -0.00351088256496619 - 0.1452662186434*I,
 -0.00351088256496619 + 0.1452662186434*I]

In [51]:
import pandas as pd

df = pd.read_csv('/Users/katerynaivaniuk/Downloads/Bridge Reconstructions - Sheet1 (2).csv')
df


,Bridge ID,Bridge Name,Region,Bridge Function,Reconstruction Costs,Volume (Num of vehicles per day on average),Total Area of the Damage,Damage Level,Length of the bridge damaged (m),Width of the bridge damaged (m)
0,ID004,Bakhmut Rail Bridge,Donetsk,Railway,"556,311.00","3,200",185.437,L4,41.3,4.49
1,ID013,Siverskyi Donetsk River Bridge,Donetsk,Railway,"1,887,840.00","6,800",629.280,L4,114.0,5.52
2,ID021,Bohorodychne Bridge,Donetsk,Highway,"4,176,000.00","8,700",1392.000,L4,120.0,11.60
3,ID015,Staryi Saltiv Dam Bridge,Kharkiv,Highway,"710,964.00","8,000",236.988,L4,34.8,6.81
4,ID016,Rus'ka Lozova Bridge,Kharkiv,Railway,"1,972,860.00",120,657.620,L4,50.2,13.10
5,ID020,Horokhovatka Bridge,Kharkiv,Highway,"1,326,960.00","3,500",442.320,L4,58.2,7.60
6,ID001,Antonivsky Bridge,Kherson,Highway,"6,825,000.00","7,600",2275.000,L4,91.0,25.00
7,ID005,Chongar Bridge,Kherson,Highway,"963,360.00","8,100",321.120,L3,22.3,14.40
8,ID009,Darivka Bridge,Kherson,Highway,"6,129,900.00",9400,2043.300,L4,139.0,14.70
9,ID011,Henichesk Bridge,Kherson,Highway and Railway,"719,550.00","5,300",239.850,L3,19.5,12.30


In [52]:
print(df.columns)

Index(['Bridge ID ', 'Bridge Name ', 'Region', 'Bridge Function',
       'Reconstruction Costs', 'Volume (Num of vehicles per day on average) ',
       'Total Area of the Damage ', 'Damage Level ',
       'Length of the bridge damaged (m)', 'Width of the bridge damaged (m)'],
      dtype='object')


In [53]:
df['Bridge Function'] = df['Bridge Function'].str.strip()
df['Bridge Function'] = df['Bridge Function'].replace({"Highway and Railway": "Highway"})

# Mapping bridge function (highway vs. railway) to numeric values
df['Bridge Function'] = df['Bridge Function'].map({'Highway': 1, 'Railway': 0})

print(df['Bridge Function'])


0     0.0
1     0.0
2     1.0
3     1.0
4     0.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    0.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    NaN
23    NaN
24    NaN
Name: Bridge Function, dtype: float64


In [54]:
import numpy as np

def ahp_pairwise_comparison_matrix():
    return np.array([
        [1, 7, 3],   
        [1/7, 1, 1/3],  
        [1/3, 3, 1]   
    ])

def calculate_ahp_weights(pairwise_matrix):
    # Normalizing and calculating weights (priority vector) for each criterion
    column_sums = pairwise_matrix.sum(axis=0)
    normalized_matrix = pairwise_matrix / column_sums
    priority_vector = normalized_matrix.mean(axis=1)

    # Consistency check
    weighted_sum = np.dot(pairwise_matrix, priority_vector)
    lambda_max = (weighted_sum / priority_vector).mean()
    CI = (lambda_max - len(pairwise_matrix)) / (len(pairwise_matrix) - 1)
    RI = 0.58  # for 3x3 matrix
    CR = CI / RI

    if CR < 0.1:
        return priority_vector
    else:
        raise ValueError("The pairwise matrix is inconsistent (CR > 0.1). Please adjust the values.")

# Applying AHP Algorithm Within Each Region
results = []

for region, group in df.groupby("Region"):
    if len(group) > 1:
        volume = pd.to_numeric(group['Volume (Num of vehicles per day on average) '].replace({',': ''}, regex=True))
        cost = pd.to_numeric(group['Reconstruction Costs'].replace({',': '', '€': ''}, regex=True))
        function = group['Bridge Function']
        
        # Normalizing Volume and Cost within the region
        total_volume = volume.sum()
        total_cost = cost.sum()

        normalized_volume = volume / total_volume
        normalized_cost = cost / total_cost

        # Getting weights for Volume, Cost, and Function
        pairwise_matrix = ahp_pairwise_comparison_matrix()
        weights = calculate_ahp_weights(pairwise_matrix)

        # Calculating AHP score for each bridge in the region
        scores = (normalized_volume * weights[0]) + (normalized_cost * weights[1]) + (function * weights[2])

        region_results = pd.DataFrame({
            'Region': region,
            'Bridge ID': group['Bridge ID '],
            'AHP Score': scores
        }).sort_values(by='AHP Score', ascending=False)
        
        results.append(region_results)

for region_results in results:
    region = region_results['Region'].iloc[0]  # Get the region name
    print(f"\nPrioritization for Region: {region}")
    for _, row in region_results.iterrows():
        print(f"Bridge {row['Bridge ID']} with AHP Score: {row['AHP Score']:.4f}")



Prioritization for Region: Donetsk 
Bridge ID021 with AHP Score: 0.6098
Bridge ID013 with AHP Score: 0.2683
Bridge ID004 with AHP Score: 0.1218

Prioritization for Region: Kharkiv 
Bridge ID015 with AHP Score: 0.7191
Bridge ID020 with AHP Score: 0.4737
Bridge ID016 with AHP Score: 0.0503

Prioritization for Region: Kherson
Bridge ID009 with AHP Score: 0.4868
Bridge ID001 with AHP Score: 0.4514
Bridge ID005 with AHP Score: 0.4271
Bridge ID011 with AHP Score: 0.3640

Prioritization for Region: Luhansk
Bridge ID017 with AHP Score: 0.5708
Bridge ID010 with AHP Score: 0.4770
Bridge ID019 with AHP Score: 0.3299
Bridge ID007 with AHP Score: 0.3048
Bridge ID018 with AHP Score: 0.0468

Prioritization for Region: Zaporizhzhia 
Bridge ID014 with AHP Score: 0.7041
Bridge ID002 with AHP Score: 0.5390
